<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Kleines-Beispiel" data-toc-modified-id="Kleines-Beispiel-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Kleines Beispiel</a></span></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [80]:
from copy import copy
import nglview as nv
import MDAnalysis as mda
import numpy as np
import time
import ipywidgets as widgets
from scipy.sparse import csr_array

from molgri.paths import PATH_OUTPUT_PT, OUTPUT_PLOTTING_DATA, PATH_OUTPUT_LOGGING
from molgri.space.fullgrid import FullGrid

Erstmals: wir brauchen eine HF Monomer Datai.

In [3]:
%pycat input/HF.xyz

Nun lass zwei PTs (klein, groß) produzieren:
- klein: -o 42 -t "linspace(0.1, 0.4, 10)" -b 8
- groß: -o 162 -t "linspace(0.1, 0.4, 20)" -b 40

## Kleines Beispiel
- obtain full grid
- show full grid coordinates per frame in the corner

In [46]:
small_name = "HF_HF_0010"
b_grid = "8"
o_grid = "12"
t_grid = "linspace(0.2, 0.4, 3)"

In [47]:
%run molgri/scripts/generate_pt.py -m1 HF -m2 HF --recalculate -o $o_grid -b $b_grid -t "linspace(0.2, 0.4, 3)"

Saved the log file to output/data/logging/HF_HF_0013.log
Saved pseudo-trajectory to output/data/pt_files/HF_HF_0013.xtc and structure file to output/data/pt_files/HF_HF_0013.gro
Timing the generation of Pt HF_HF_0013: 0:00:00.589081 hours:minutes:seconds


In [ ]:
u = mda.Universe(f"output/data/pt_files/{small_name}.gro", f"output/data/pt_files/{small_name}.xtc")

w = nv.show_mdanalysis(u, gui=True)
for frame in range(15):
    w.shape.add('text', [0, 4, -1], [0.2, 0.5, 0.8], 2.5, np.array2string(full_coordinates[frame], precision=2, 
                                                                          suppress_small=True, separator=","))

In [ ]:
w

In [ ]:
for frame in range(15):
    #w.clear_representations()
    w.add_representation(repr_type="label", name = "label", showBackground = True, labelType = "atomname", color = "black", 
                    xOffset = 0.5 , zOffset =5 )
    view.shape.add_label("HelloWorld")
    w.update_representation(repr_index=-1)
    time.sleep(1)
    
    w.frame = frame
w

In [114]:
class PTFrameAnalyser:
    
    """
    Visualise frames and important information from PT using NGLView. Reads exclusively from saved files.
    """
    
    def __init__(self, name_pt: str, o_input, b_input, t_input: str):
        """
        Args:
            - name_pt: Name that is given to the PT files, usually in the form <molecule 1>_<molecule 2>_<4-number index>
                       or selected by the user
            - name_grid: Automatically generated name of the full grid in the form like
                         b_cube4D_<number>_o_ico_<number>_t_<hash>
        """
        
        self.name_pt = name_pt

        
        self.u = mda.Universe(f"{PATH_OUTPUT_PT}{self.name_pt}.gro", f"{PATH_OUTPUT_PT}{self.name_pt}.xtc")
        self.fg = FullGrid(o_grid_name=o_input, b_grid_name=b_input, t_grid_name=t_input, use_saved=True)
    
    def _get_neig_indices(self, i):
        adj_array = csr_array(self.fg.get_full_adjacency())[:, [i]].toarray().T[0]
        neighbour_indices = np.nonzero(adj_array)[0]
        return neighbour_indices
        
    
    def plot_all_neighours_of_i(self, i: int):
        neighbour_indices = self._get_neig_indices(i)
        
        box = HBox(layout=Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
        box.overflow_x = 'auto'
        
        all_views = []
        for neig_i in neighbour_indices:
            neig_view = copy(self.plot_ith_frame(neig_i))
            # lightly plot the original structure
            neig_view = copy(self.plot_ith_frame(i, opacity=0.5, view=neig_view))
            neig_view.layout.width = "200px"
            
            all_views.append(neig_view)
        
        # sync all views
        for v in all_views:
            v._set_sync_camera(all_views)
        
        box.children=[i for i in all_views]
        display(box)
        
                
    def _get_fullgrid_name(self):
        """
        NOT IN USE CURRENTLY
        Read from the .log file to obtain the name of the used FullGrid.
        """
        with open(f"{PATH_OUTPUT_LOGGING}{self.name_pt}.log") as f:
            while True:
                line = f.readline()
                if line.startswith("INFO:PtLogger:full grid name:"):
                    return line.strip().split(": ")[-1]
        
    
    def get_ith_frame(self, i: int) -> mda.Universe:
        """
        Get the Universe object containing of all the atoms in the PT but only a single frame (i-th frame).
        """
        self.u.trajectory[i]
        all_atoms = self.u.select_atoms('all')
        new_u = mda.Merge(self.u.atoms)
        return new_u
    
    def plot_all(self, views: list, n_columns: int = 2):
        """
        Plot a list of NGLView Views in several columns and rows.
        """
        #num_rows=len(views)//n_columns
        
        #hwidgets = []
        #for row in range(num_rows):
            
        #    hwidgets.append(widgets.HBox(views[row:row+n_columns]))
        
        #vbox = widgets.VBox(hwidgets)
        print(type(views), views)
        gridbox = widgets.GridBox(views)
        display(gridbox)
    
    def plot_side_by_side(self, view1, view2):
        """
        Plot two NGLView views side by side.
        
        """
        # Adjust layout to center both molecules
        view1.layout.width = 'auto'
        view2.layout.width = 'auto'
        view1._set_sync_camera([view2])
        view2._set_sync_camera([view1])


        hbox = widgets.HBox([view1, view2])
        
                
        # TODO: link cameras of both views
        mylink = widgets.jslink((view1, 'frame'), (view2, 'frame'))

        # Display the HBox
        display(hbox)

    
    def _add_coordinate_axes(self, view):
        """
        Add the x, y and z axes at origin to a NGLView.
        """
        
        # arguments of add_arrow are: start position, end position, color (in RGB), radius of arrow head
        # arguments of add_label are: position, color (in RGB), size, text
        
        # X-axis is red
        view.shape.add_arrow([0, 0, 0], [1, 0, 0],[ 1, 0, 0 ], 0.1)
        view.shape.add_label([1, 0, 0], [1, 0, 0], 1.5, 'x')
        
        # Y-axis is green
        view.shape.add_arrow([0, 0, 0], [0, 1, 0],[0, 1, 0 ], 0.1)
        view.shape.add_label([0, 1, 0], [0, 1, 0], 1.5, 'y')
        
        # Z-axis is blue
        view.shape.add_arrow([0, 0, 0], [0, 0, 1],[0, 0, 1 ], 0.1)
        view.shape.add_label([0, 0, 1], [0, 0, 1], 1.5, 'z')

    def plot_ith_frame(self, i: int, view=None, axes=True, **kwargs):
        """
        Plot i-th frame of PT including the frame index, 7D coordinates
        """
        ith_atoms = self.get_ith_frame(i)
        
        if view is None:
            view = nv.show_mdanalysis(ith_atoms, **kwargs)
        else:
            view.add_trajectory(ith_atoms, **kwargs)
        if axes:
            self._add_coordinate_axes(view)
        
        return view
    
    def print_ith_information(self, i):
        all_points = self.fg.get_full_grid_as_array()
        ith_fg_coo = np.array2string(all_points[i], precision=2, suppress_small=True, separator=",")
        print(f"Frame {i}")
        print(f"FullGrid coordinates {ith_fg_coo}")
        
        neighbours = self._get_neig_indices(i)
        print(f"The point {i} has {len(neighbours)} neighbours, here is the list:")
        for n_index in neighbours:
            print(np.array2string(all_points[n_index], precision=2, suppress_small=True, separator=","))
        

pta = PTFrameAnalyser(small_name, b_input=b_grid, o_input=o_grid, t_input=t_grid)
#view1 = pta.plot_ith_frame(15)
#view2 = pta.plot_ith_frame(75)
#pta.plot_side_by_side(view1, view2)
pta.print_ith_information(5)
pta.plot_all_neighours_of_i(5)

Frame 5
FullGrid coordinates [-1.7 , 0.  , 1.05, 0.5 , 0.5 , 0.5 ,-0.5 ]
The point 5 has 10 neighbours, here is the list:
[-1.7 , 0.  , 1.05, 0.5 ,-0.5 ,-0.5 , 0.5 ]
[-1.7 , 0.  , 1.05, 0.5 ,-0.5 , 0.5 ,-0.5 ]
[-1.7 , 0.  , 1.05, 0.5 , 0.5 , 0.5 , 0.5 ]
[-1.7 , 0.  , 1.05, 0.5 , 0.5 ,-0.5 ,-0.5 ]
[-1.05,-1.7 , 0.  , 0.5 , 0.5 , 0.5 ,-0.5 ]
[ 0.  ,-1.05, 1.7 , 0.5 , 0.5 , 0.5 ,-0.5 ]
[-1.05, 1.7 , 0.  , 0.5 , 0.5 , 0.5 ,-0.5 ]
[-1.7 , 0.  ,-1.05, 0.5 , 0.5 , 0.5 ,-0.5 ]
[ 0.  , 1.05, 1.7 , 0.5 , 0.5 , 0.5 ,-0.5 ]
[-2.55, 0.  , 1.58, 0.5 , 0.5 , 0.5 ,-0.5 ]
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}
{'opacity': 0.5}


In [ ]:
view1 = pta.plot_ith_frame(15)
view2 = pta.plot_ith_frame(75)

In [ ]:
pta.plot_side_by_side(view1, view2)

In [ ]:
pta.plot_all([view1, view2], 1)

- plot neighbours
- plot structures with largest phasespace volumes

In [89]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display

num_checks = 50

cb_cont = HBox(layout=Layout(width='100%',display='inline-flex',flex_flow='row wrap'))
#cb_cont.overflow_x = 'scroll'

checkboxes = []
for f in range(num_checks):
    checkboxes.append(\
        widgets.Checkbox(description = 'Checkbox [%d]'%(f), \
                     value=False, width=50))

cb_cont.children=[i for i in checkboxes]
display(cb_cont)
